In [102]:
%defaultDatasource jdbc:h2:mem:db

# SQL

# Base de dados e proposta
A base escolhida para essa análise foi a unipro(https://www.uniprot.org/diseases/), uma das bases cuja a qual virushost (https://www.genome.jp/virushostdb/) obtem seus dados.


Optamos pela sub base UniProtKB pois ela contem dados mais específicos sobre doenças humanas nas quais as proteínas estão envolvidas 

# Criando tabela para analise



In [103]:
DROP TABLE IF EXISTS Diseases;
CREATE TABLE Diseases (
  Id VARCHAR(8) NOT NULL,
  Name VARCHAR(150) ,
  Mnemonic VARCHAR(150) ,
  Description VARCHAR(2000),
  PRIMARY KEY(Id)               
) AS SELECT
  ID,Name,Mnemonic,Description
FROM CSVREAD('diseases-all.csv',null, 'UTF-8', CHAR(9));

SELECT * FROM Diseases;

## Criando e padronizando tabela para análise

In [165]:
DROP TABLE IF EXISTS DiseasesAnalysis;
CREATE TABLE DiseasesAnalysis (
  Category VARCHAR(150),
  Amount INTEGER);
  
DROP TABLE IF EXISTS DiseasesAnalysisAux;
CREATE TABLE DiseasesAnalysisAux (
  Category VARCHAR(150),
  Amount INTEGER);
 
INSERT INTO DiseasesAnalysisAux SELECT LOWER (Category), Count(*) Amount 
                                 FROM  (SELECT SUBSTRING(name, 1, CHARINDEX(',', name) - 1) Category FROM Diseases
                                 GROUP BY Category) 
                                 WHERE Category NOT LIKE ''
                                 GROUP BY Category; 
                                 
INSERT INTO DiseasesAnalysisAux SELECT LOWER (name), Count(*) Amount 
                                 FROM  Diseases
                                 WHERE CHARINDEX(',', name) = 0
                                 GROUP BY name;   

UPDATE DiseasesAnalysisAux  
SET Category = SUBSTRING(Category, 1, LENGTH(Category)-3) 
WHERE SUBSTRING(Category, LENGTH(Category)-2, 1) IN ('0','1','2','3','4','5','6','7','8','9', ' ');

UPDATE DiseasesAnalysisAux  
SET Category = SUBSTRING(Category, 1, LENGTH(Category)-2) 
WHERE SUBSTRING(Category, LENGTH(Category)-1, 1) IN ('0','1','2','3','4','5','6','7','8','9', ' ');

UPDATE DiseasesAnalysisAux  
SET Category = SUBSTRING(Category, 1, LENGTH(Category)-1) 
WHERE SUBSTRING(Category, LENGTH(Category), 1) IN ('0','1','2','3','4','5','6','7','8','9', ' ', '-');

DELETE FROM DiseasesAnalysisAux
WHERE Category = ' ' OR
      Category = '';
      

INSERT INTO DiseasesAnalysis SELECT Category, Count(*) Amount 
                             FROM  DiseasesAnalysisAux 
                             GROUP BY Category;

DROP TABLE DiseasesAnalysisAux;

INSERT INTO DiseasesAnalysis VALUES ('deficiency',(SELECT COUNT(*) FROM Diseases WHERE name LIKE '%deficiency%'));
INSERT INTO DiseasesAnalysis VALUES ('congenital',(SELECT COUNT(*) FROM Diseases WHERE name LIKE '%congenital%'));
INSERT INTO DiseasesAnalysis VALUES ('sex reversal',(SELECT COUNT(*) FROM Diseases WHERE name LIKE '%sex reversal%'));
INSERT INTO DiseasesAnalysis VALUES ('insufficiency',(SELECT COUNT(*) FROM Diseases WHERE name LIKE '%insufficiency%'));
INSERT INTO DiseasesAnalysis VALUES ('syndrome',(SELECT COUNT(*) FROM Diseases WHERE name LIKE '%syndrome%'));
INSERT INTO DiseasesAnalysis VALUES ('autosomal recessive',(SELECT COUNT(*) FROM Diseases WHERE name LIKE '%autosomal recessive%'));
INSERT INTO DiseasesAnalysis VALUES ('autosomal dominant',(SELECT COUNT(*) FROM Diseases WHERE name LIKE '%autosomal dominant%'));
INSERT INTO DiseasesAnalysis VALUES ('hereditary',(SELECT COUNT(*) FROM Diseases WHERE name LIKE '%hereditary%'));
INSERT INTO DiseasesAnalysis VALUES ('mitochondrial',(SELECT COUNT(*) FROM Diseases WHERE name LIKE '%mitochondrial%'));
INSERT INTO DiseasesAnalysis VALUES ('juvenile',(SELECT COUNT(*) FROM Diseases WHERE name LIKE '%juvenile%'));
INSERT INTO DiseasesAnalysis VALUES ('X-linked',(SELECT COUNT(*) FROM Diseases WHERE name LIKE '%X-linked%'));
INSERT INTO DiseasesAnalysis VALUES ('Y-linked',(SELECT COUNT(*) FROM Diseases WHERE name LIKE '%Y-linked%'));                                                 
INSERT INTO DiseasesAnalysis VALUES ('dysplasia',(SELECT COUNT(*) FROM Diseases WHERE name LIKE '%dysplasia%'));
INSERT INTO DiseasesAnalysis VALUES ('infantile',(SELECT COUNT(*) FROM Diseases WHERE name LIKE '%infantile%'));
INSERT INTO DiseasesAnalysis VALUES ('encephalopathy',(SELECT COUNT(*) FROM Diseases WHERE name LIKE '%encephalopathy%'));
INSERT INTO DiseasesAnalysis VALUES ('sarcoma',(SELECT COUNT(*) FROM Diseases WHERE name LIKE '%sarcoma%'));
INSERT INTO DiseasesAnalysis VALUES ('mental',(SELECT COUNT(*) FROM Diseases WHERE name LIKE '%mental%'));
INSERT INTO DiseasesAnalysis VALUES ('deafness',(SELECT COUNT(*) FROM Diseases WHERE name LIKE '%deafness%'));
                                               
                                               
select * from DiseasesAnalysis order by Category;

# Algumas pesquisas interessantes em relação a essa tabela

## Proposta 1: Doenças mais comuns
Esta query em SQL, retorna as informações das 5 categorias de doenças mais comuns relacionadas a algum tipo de proteina

In [167]:
SELECT * FROM DiseasesAnalysis
ORDER BY amount DESC
LIMIT 11;

## Proposta 2:Suporte
Esta query em SQL, retorna o valor do suporte de cada categoria de doença, ou seja, a probabilidade de dada uma doença qualquer, ela ser dessa categoria específica

In [168]:
SELECT Category, SUM(CAST(Amount AS DOUBLE) / (SELECT COUNT(*) numberofdiseases
                                               FROM Diseases)) suporte
FROM DiseasesAnalysis
GROUP BY category
ORDER BY suporte DESC;

## Proposta 3: Duas categorias na mesma doença

Uma informação interessante a se pensar com relação aos dados de doençar é a quantidade de doenças que pertencem a duas categorias simultaneamente. Essa query mostra essas ocorrências pra algumas das categorias mais comuns escolhidas para analise

In [169]:
SELECT 'syndrome', 'deficiency', COUNT(*) occurrences
FROM Diseases D
WHERE D.name LIKE '%syndrome%' AND
      D.name LIKE '%deficiency%';
      
SELECT 'syndrome', 'mental', COUNT(*) occurrences
FROM Diseases D
WHERE D.name LIKE '%syndrome%' AND
      D.name LIKE '%mental%';
      
SELECT 'syndrome', 'congenital', COUNT(*) occurrences
FROM Diseases D
WHERE D.name LIKE '%syndrome%' AND
      D.name LIKE '%congenital%';
      
SELECT 'mental', 'deficiency', COUNT(*) occurrences
FROM Diseases D
WHERE D.name LIKE '%mental%' AND
      D.name LIKE '%deficiency%';
      
SELECT 'autosomal dominant', 'deafness', COUNT(*) occurrences
FROM Diseases D
WHERE D.name LIKE '%autosomal dominant%' AND
      D.name LIKE '%deafness%';
      
SELECT 'autosomal dominant', 'syndrome', COUNT(*) occurrences
FROM Diseases D
WHERE D.name LIKE '%autosomal dominant%' AND
      D.name LIKE '%syndrome%';
      
SELECT 'autosomal dominant', 'mental', COUNT(*) occurrences
FROM Diseases D
WHERE D.name LIKE '%autosomal dominant%' AND
      D.name LIKE '%mental%';
      
SELECT 'X-linked', 'dysplasia', COUNT(*) occurrences
FROM Diseases D
WHERE D.name LIKE '%X-linked%' AND
      D.name LIKE '%dysplasia%';
      
SELECT 'hereditary', 'syndrome', COUNT(*) occurrences
FROM Diseases D
WHERE D.name LIKE '%hereditary%' AND
      D.name LIKE '%syndrome%';

## Proposta 4: Confiança

Outra informação possivelmente mais importante que as ocorrências de duas categorias ao mesmo tempo, é a confiança dessas associações: a probabilidade de que ambas ocorram ao mesmo tempo

In [170]:
SELECT 'syndrome', 'deficiency', CAST((SELECT COUNT(*) occurrences
                                       FROM Diseases D
                                       WHERE D.name LIKE '%syndrome%' AND
                                             D.name LIKE '%deficiency%') AS DOUBLE) / (SELECT amount
                                                                                       FROM DiseasesAnalysis
                                                                                       WHERE category = 'syndrome') confiança;
                                                             
SELECT 'syndrome', 'mental', CAST((SELECT COUNT(*) occurrences
                                   FROM Diseases D
                                   WHERE D.name LIKE '%syndrome%' AND
                                         D.name LIKE '%mental%') AS DOUBLE) / (SELECT amount
                                                                               FROM DiseasesAnalysis
                                                                               WHERE category = 'syndrome') confiança;
                                                             
SELECT 'syndrome', 'congenital', CAST((SELECT COUNT(*) occurrences
                                       FROM Diseases D
                                       WHERE D.name LIKE '%syndrome%' AND
                                             D.name LIKE '%congenital%') AS DOUBLE) / (SELECT amount
                                                                                       FROM DiseasesAnalysis
                                                                                       WHERE category = 'syndrome') confiança;
                                                             
SELECT 'mental', 'deficiency', CAST((SELECT COUNT(*) occurrences
                                     FROM Diseases D
                                     WHERE D.name LIKE '%mental%' AND
                                           D.name LIKE '%deficiency%') AS DOUBLE) / (SELECT amount
                                                                                     FROM DiseasesAnalysis
                                                                                     WHERE category = 'mental') confiança;
                                                             
SELECT 'autosomal dominant', 'deafness', CAST((SELECT COUNT(*) occurrences
                                               FROM Diseases D
                                               WHERE D.name LIKE '%autosomal dominant%' AND
                                                     D.name LIKE '%deafness%') AS DOUBLE) / (SELECT amount
                                                                                             FROM DiseasesAnalysis
                                                                                             WHERE category = 'autosomal dominant') confiança;
                                                             
SELECT 'autosomal dominant', 'syndrome', CAST((SELECT COUNT(*) occurrences
                                               FROM Diseases D
                                               WHERE D.name LIKE '%autosomal dominant%' AND
                                                     D.name LIKE '%syndrome%') AS DOUBLE) / (SELECT amount
                                                                                             FROM DiseasesAnalysis
                                                                                             WHERE category = 'autosomal dominant') confiança;
                                                             
SELECT 'autosomal dominant', 'mental', CAST((SELECT COUNT(*) occurrences
                                             FROM Diseases D
                                             WHERE D.name LIKE '%autosomal dominant%' AND
                                                   D.name LIKE '%mental%') AS DOUBLE) / (SELECT amount
                                                                                         FROM DiseasesAnalysis
                                                                                         WHERE category = 'autosomal dominant') confiança;
                                                             
SELECT 'X-linked', 'dysplasia', CAST((SELECT COUNT(*) occurrences
                                      FROM Diseases D
                                      WHERE D.name LIKE '%X-linked%' AND
                                            D.name LIKE '%dysplasia%') AS DOUBLE) / (SELECT amount
                                                                                     FROM DiseasesAnalysis
                                                                                     WHERE category = 'X-linked') confiança;

## Proposta 5: Probabilidade

Outra informação que podemos retirar comparar as probabilidade de uma doença ser de uma categoria mais especifica em relação a uma menos especifica.

In [171]:
SELECT CAST(SELECT SUM(amount) from DiseasesAnalysis 
            WHERE category LIKE '%autosomal dominant%' AS DOUBLE)/
            CAST((SELECT count(*) NAME 
                  FROM Diseases 
                  WHERE name LIKE '%autosomal%') AS DOUBLE) autosomalDominant_autosomal,
        CAST(SELECT SUM(amount) from DiseasesAnalysis 
            WHERE category LIKE '%autosomal dominant%' AS DOUBLE)/
            CAST((SELECT count(*) NAME 
                  FROM Diseases) AS DOUBLE) autosomalDominant_total;

SELECT (CAST(SELECT SUM(amount) from DiseasesAnalysis 
            WHERE category LIKE '%autosomal recessive%' AS DOUBLE)/
            CAST((SELECT count(*) NAME 
                  FROM Diseases 
                  WHERE name LIKE '%autosomal%') AS DOUBLE))autosomalRecessive_autosomal,
       (CAST(SELECT SUM(amount) from DiseasesAnalysis 
            WHERE category LIKE '%autosomal recessive%' AS DOUBLE)/
            CAST((SELECT count(*) NAME 
                  FROM Diseases) AS DOUBLE)) autosomalRecessive_total;

SELECT (CAST(SELECT SUM(amount) from DiseasesAnalysis 
            WHERE category LIKE '%x-linked%' AS DOUBLE)/
            CAST((SELECT count(*) NAME 
                  FROM Diseases 
                  WHERE name LIKE '%linked%') AS DOUBLE)) X_LINKED,
       (CAST(SELECT SUM(amount) from DiseasesAnalysis 
            WHERE category LIKE '%x-linked%' AS DOUBLE)/
            CAST((SELECT count(*) NAME 
                  FROM Diseases) AS DOUBLE)) X_TOTAL;
                  
SELECT (CAST(SELECT SUM(amount) from DiseasesAnalysis 
            WHERE category LIKE '%y-linked%' AS DOUBLE)/
            CAST((SELECT count(*) NAME 
                  FROM Diseases 
                  WHERE name LIKE '%linked%') AS DOUBLE)) Y_LINKED,
       (CAST(SELECT SUM(amount) from DiseasesAnalysis 
            WHERE category LIKE '%y-linked%' AS DOUBLE)/
            CAST((SELECT count(*) NAME 
                  FROM Diseases) AS DOUBLE)) Y_TOTAL;
